## GMRES

### Why do we care?

## Nonlinear least square

**Theorem** (Vector dot product rule)

Let $u\left(x_1, \ldots, x_n\right)$ and $v\left(x_1, \ldots, x_n\right)$ be $\mathbb{R}^n$-vector-valued functions, and let $A\left(x_1, \ldots, x_n\right)$ be an $n \times n$ matrix function. The dot product $u^T v$ is a scalar function. Then, we have


$$
\nabla\left(u^T v\right)=v^T D u+u^T D v,
$$

and

$$
D(A v)=A \cdot D v+\sum_{i=1}^n v_i D a_i,
$$

where $a_i$ denotes the $i$ th column of $A$.

### Appendix

---
This work is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)